<a href="https://colab.research.google.com/github/Ranjith-A13/Python/blob/main/Recommendation_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv("anime.csv")
print(df.head())
print(df.info())

   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  

In [6]:
# Fill missing genres with 'Unknown'
df['genre'] = df['genre'].fillna('Unknown')

# Fill missing ratings with the mean rating
df['rating'] = df['rating'].fillna(df['rating'].mean())

# Convert episodes to numeric and fill missing with median
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')
df['episodes'] = df['episodes'].fillna(df['episodes'].median())

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# TF-IDF on genres
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['genre'])

# Normalize ratings
scaler = MinMaxScaler()
rating_scaled = scaler.fit_transform(df[['rating']])

# Combine features (TF-IDF genre + rating)
from scipy.sparse import hstack
features = hstack([tfidf_matrix, rating_scaled])

# Compute cosine similarity
cosine_sim = cosine_similarity(features, features)

# Function to recommend anime
def recommend_anime(title, cosine_sim=cosine_sim, df=df, top_n=5):
    idx = df[df['name'] == title].index
    if len(idx) == 0:
        return "Anime not found."
    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # Skip the input anime itself
    anime_indices = [i[0] for i in sim_scores]
    return df['name'].iloc[anime_indices]

# Example usage
print(recommend_anime("Naruto"))


615                                    Naruto: Shippuuden
1103    Boruto: Naruto the Movie - Naruto ga Hokage ni...
486                              Boruto: Naruto the Movie
1343                                          Naruto x UT
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
Name: name, dtype: object
